In [51]:
include("AuxiliaryFunctions/BornModel.jl")
include("AuxiliaryFunctions/Plotting.jl")

import .BornModel as BM
import .Plotting as P
using Plots, Plots.PlotMeasures
using DifferentialEquations
using BifurcationKit, Parameters

In [61]:
# plot settings

linewidth = 1.5
titlefontsize = 12 
legendfontsize = 10

10

## S2

In [2]:
parsS2 = (T2 = BM.nondimensional_T(10.), S2 = BM.nondimensional_S(34.), T4 = BM.nondimensional_T(4.), S4 = BM.nondimensional_S(34.9), T0 = BM.nondimensional_T(1.6), Tamp = 0,
    η = 1.29e2, μ1 = 0.838, μ2 = 83.8, μ3 = 12.1, μ4 = 8.84e-3, r = 0.0714)

x0S2 = [BM.nondimensional_T(6), BM.nondimensional_S(34.5), BM.nondimensional_T(5), BM.nondimensional_S(34.9)]
tspan = (0., 50.)

odeprobS2 = ODEProblem(BM.nondimensional_born!, x0S2, tspan, parsS2)
odesolS2 = solve(odeprobS2)

x0S2 = odesolS2[end];

In [3]:
recordFromSolution(x, p) = (T1 = x[1], S1 = x[2], T3 = x[3], S3 = x[4],
    M = parsS2.r*(1 - parsS2.η*((parsS2.S4 - 1) - (x[4] - x[3])) - parsS2.η*parsS2.r*((p - parsS2.T2) - (x[2] - x[1]))) + (1 - parsS2.η*((parsS2.S4 - 1) - (x[4] - x[3]))),
    Δσ13 = (x[2] - x[1]) - (x[4] - x[3]),
    Δσ12 = (p  - parsS2.T2) - (x[2] - x[1]),
    Δσ34 = (parsS2.S4  - parsS2.T4) - (x[4] - x[3]),
    Ud = 1 - parsS2.η*((parsS2.S4 - 1) - (x[4] - x[3])),
    Us = 1 - parsS2.η*((parsS2.S4 - 1) - (x[4] - x[3])) - parsS2.η*parsS2.r*((p - parsS2.T2) - (x[2] - x[1]))
    )

probS2 = BifurcationProblem(BM.nondimensional_born!, x0S2, parsS2, (@lens _.S2); 
    record_from_solution = recordFromSolution
)    

# continuation
opts_newton_c = NewtonPar(tol = 1e-11, max_iterations = 50)
opts_c = ContinuationPar(p_min = BM.nondimensional_S(0), p_max = BM.nondimensional_S(50),
    dsmin=1e-10, dsmax=1e-5, ds=1e-6,
    newton_options = opts_newton_c,
    max_steps = 5e4)

brS2 = continuation(probS2, PALC(), opts_c; 
    detect_bifurcation = 3, 
    bothside = true,
    normC = norminf,
    usedeflation = true)

# diagram
opts_newton_d = NewtonPar(tol = 1e-11, max_iterations = 50)
opts_d = ContinuationPar(p_min = BM.nondimensional_S(0), p_max = BM.nondimensional_S(50),
    dsmin=1e-10, dsmax=1e-6, ds=1e-7,
    newton_options = opts_newton_d,
    max_steps = 1e6) 

dS2 = bifurcationdiagram(probS2, PALC(),
    5,
    bothside = true,
    (args...) -> opts_d,
)

[Bifurcation diagram]
 ┌─ From 0-th bifurcation point.
 ├─ Children number: 0
 └─ Root (recursion level 1)
      ┌─ Curve type: EquilibriumCont
      ├─ Number of points: 1678901
      ├─ Type of vectors: Vector{Float64}
      ├─ Parameter S2 starts at 0.0, ends at 0.01265963541810787
      ├─ Algo: PALC
      └─ Special points:

If `br` is the name of the branch,
ind_ev = index of the bifurcating eigenvalue e.g. `br.eig[idx].eigenvals[ind_ev]`

- #  1, endpoint at S2 ≈ +0.00000000,                                                                      step =   0
- #  2,       bp at S2 ≈ +0.88400540 ∈ (+0.88400540, +0.88400540), |δp|=1e-10, [converged], δ = ( 1,  0), step = 698879, eigenelements in eig[698880], ind_ev =   1
- #  3,       bp at S2 ≈ +0.85751433 ∈ (+0.85751433, +0.85751433), |δp|=5e-11, [converged], δ = ( 1,  0), step = 717737, eigenelements in eig[717738], ind_ev =   2
- #  4,     hopf at S2 ≈ +0.85759615 ∈ (+0.85759613, +0.85759615), |δp|=2e-08, [converged], δ = (-2, -2)

In [62]:
S2_br_lower_stable = dS2.γ.branch[1:698879]
S2_br_lower_unstable = dS2.γ.branch[698880:717802]
S2_br_upper_stable = dS2.γ.branch[717803:858237]
S2_br_upper_unstable = dS2.γ.branch[858238:end];

pS2_stable_xvalues1, pS2_stable_xvalues2 = BM.dimensional_S(S2_br_lower_stable.param), BM.dimensional_S(S2_br_upper_stable.param)
pS2_unstable_xvalues1, pS2_unstable_xvalues2 = BM.dimensional_S(S2_br_lower_unstable.param), BM.dimensional_S(S2_br_upper_unstable.param)

pS2_stable_yvalues1, pS2_stable_yvalues2 = BM.dimensional_M(S2_br_lower_stable.M), BM.dimensional_M(S2_br_upper_stable.M)
pS2_unstable_yvalues1, pS2_unstable_yvalues2 = BM.dimensional_M(S2_br_lower_unstable.M), BM.dimensional_M(S2_br_upper_unstable.M)

# plot
pS2 = plot(pS2_stable_xvalues1, pS2_stable_yvalues1, label = "lower stable branch", dpi = 300, linewidth = linewidth, 
    legend = :topleft, legendfontsize = legendfontsize, color = :dodgerblue, ls = :solid, grid = false, titlefontsize = titlefontsize, xticks = 33.5:0.5:35.5)
plot!(pS2_stable_xvalues2, pS2_stable_yvalues2, label = "upper stable branch", linewidth = linewidth, color = :dodgerblue, ls = :dashdot)
plot!(pS2_unstable_xvalues1, pS2_unstable_yvalues1, label = "unstable branch", linewidth = linewidth, color = :grey65)
plot!(pS2_unstable_xvalues2, pS2_unstable_yvalues2, label = "", linewidth = linewidth, color = :grey65)
P.plot_special_points(dS2.γ, :S2, :M; plot_legend = true)

xmin, xmax = 33.4, 35.5 #33.7, 35.25
ymin, ymax = 0, 32

xlims!(xmin, xmax)
ylims!(ymin, ymax)

title!("\n a) control parameter S\$_2\$")
xlabel!("\n S\$_2\$ [psu]")
ylabel!("M [Sv]")

savefig("Figures/bifdiagram-S2.pdf")

"c:\\Users\\kvdhe\\Documents\\a. Studie\\6 - MSc 2\\Thesis\\Paper\\Code\\Figures\\bifdiagram-S2.pdf"

In [5]:
# xmin, xmax = 32.9, 36
# ymin, ymax = 0, 32

# pS2 = P.plot_branch(dS2.γ, :S2, :M, plot_legend = true,
#     xlabel = "S\$_2\$ [psu]", ylabel = "M [Sv]", title = "a) Control parameter S\$_2\$", 
#     xlimits = (xmin, xmax), ylimits = (ymin, ymax))
# yticks!(0:5:35)

# savefig("Figures/bifdiagram-S2")

## F

In [6]:
parsF = (T2 = BM.nondimensional_T(10.), S2 = BM.nondimensional_S(35), T4 = BM.nondimensional_T(4.), S4 = BM.nondimensional_S(34.9), T0 = BM.nondimensional_T(1.6), Tamp = 0,
    η = 1.29e2, μ1 = 0.838, μ2 = 83.8, μ3 = 12.1, μ4 = BM.nondimensional_F(0.), r = 0.0714)

x0F = [BM.nondimensional_T(6), BM.nondimensional_S(34.5), BM.nondimensional_T(5), BM.nondimensional_S(34.9)]
tspan = (0., 30.)

odeprobF = ODEProblem(BM.nondimensional_born!, x0F, tspan, parsF)
odesolF = solve(odeprobF)

x0F = odesolF[end];

In [7]:
recordFromSolution(x, p) = (T1 = x[1], S1 = x[2], T3 = x[3], S3 = x[4],
    M = parsF.r*(1 - parsF.η*((parsF.S4 - 1) - (x[4] - x[3])) - parsF.η*parsF.r*((parsF.S2 - parsF.T2) - (x[2] - x[1]))) + (1 - parsF.η*((parsF.S4 - 1) - (x[4] - x[3]))),
    Δσ13 = (x[2] - x[1]) - (x[4] - x[3]),
    Δσ12 = (parsF.S2  - parsF.T2) - (x[2] - x[1]),
    Δσ34 = (parsF.S4  - parsF.T4) - (x[4] - x[3]),
    Ud = 1 - parsF.η*((parsF.S4 - 1) - (x[4] - x[3])),
    Us = 1 - parsF.η*((parsF.S4 - 1) - (x[4] - x[3])) - parsF.η*parsF.r*((parsF.S2 - parsF.T2) - (x[2] - x[1]))
    )

probF = BifurcationProblem(BM.nondimensional_born!, x0F, parsF, (@lens _.μ4); 
    record_from_solution = recordFromSolution
)    

# continuation
opts_newton_c = NewtonPar(tol = 1e-9, max_iterations = 50)
opts_c = ContinuationPar(p_min = BM.nondimensional_F(0), p_max = BM.nondimensional_F(50),
    dsmin=1e-10, dsmax=1e-5, ds=1e-6,
    newton_options = opts_newton_c,
    max_steps = 5e4)

brF = continuation(probF, PALC(), opts_c; 
    detect_bifurcation = 3, 
    bothside = true,
    normC = norminf,
    usedeflation = true)

# diagram
opts_newton_d = NewtonPar(tol = 1e-9, max_iterations = 50)
opts_d = ContinuationPar(p_min = BM.nondimensional_F(-10), p_max = BM.nondimensional_F(50),
    dsmin=1e-10, dsmax=1e-6, ds=1e-7,
    newton_options = opts_newton_d,
    max_steps = 5e5) 

dF = bifurcationdiagram(probF, PALC(),
    5,
    bothside = true,
    (args...) -> opts_d,
)    

[Bifurcation diagram]
 ┌─ From 0-th bifurcation point.
 ├─ Children number: 0
 └─ Root (recursion level 1)
      ┌─ Curve type: EquilibriumCont
      ├─ Number of points: 249081
      ├─ Type of vectors: Vector{Float64}
      ├─ Parameter μ4 starts at -0.08839978351073427, ends at -0.08839978351073427
      ├─ Algo: PALC
      └─ Special points:

If `br` is the name of the branch,
ind_ev = index of the bifurcating eigenvalue e.g. `br.eig[idx].eigenvals[ind_ev]`

- #  1, endpoint at μ4 ≈ -0.08839978,                                                                      step =   0
- #  2,     hopf at μ4 ≈ +0.04169556 ∈ (+0.04169556, +0.04169556), |δp|=3e-09, [converged], δ = ( 2,  2), step = 92098, eigenelements in eig[92099], ind_ev =   2
- #  3,       bp at μ4 ≈ +0.04173738 ∈ (+0.04173737, +0.04173738), |δp|=7e-09, [converged], δ = (-1,  0), step = 92133, eigenelements in eig[92134], ind_ev =   2
- #  4,       bp at μ4 ≈ +0.00883299 ∈ (+0.00883295, +0.00883299), |δp|=3e-08, [converged],

┌ Warning: The bifurcating eigenvalue is not that close to Re = 0. We found -0.10172303291052842 !≈ 0.  You can perhaps increase the argument `nev`.
└ @ BifurcationKit C:\Users\kvdhe\.julia\packages\BifurcationKit\5DxOO\src\NormalForms.jl:35


In [53]:
idx_positive = findfirst(x -> x>0, dF.γ.param)

F_br_upper_stable = dF.γ.branch[idx_positive:92098]
F_br_upper_unstable = dF.γ.branch[92099:115500]   
F_br_lower_stable = dF.γ.branch[115501:145785]
F_br_lower_unstable =  dF.γ.branch[145786:end];

pF_stable_xvalues1, pF_stable_xvalues2 = BM.dimensional_F(F_br_lower_stable.param), BM.dimensional_F(F_br_upper_stable.param)
pF_unstable_xvalues1, pF_unstable_xvalues2 = BM.dimensional_F(F_br_lower_unstable.param), BM.dimensional_F(F_br_upper_unstable.param)

pF_stable_yvalues1, pF_stable_yvalues2 = BM.dimensional_M(F_br_lower_stable.M), BM.dimensional_M(F_br_upper_stable.M)
pF_unstable_yvalues1, pF_unstable_yvalues2 = BM.dimensional_M(F_br_lower_unstable.M), BM.dimensional_M(F_br_upper_unstable.M)

# plot
pF = plot(pF_stable_xvalues1, pF_stable_yvalues1, label = "", dpi = 300, linewidth = linewidth, color = :dodgerblue, 
    ls = :solid, grid = false, legend= false, titlefontsize = titlefontsize)
plot!(pF_stable_xvalues2, pF_stable_yvalues2, label = "", linewidth = linewidth, color = :dodgerblue, ls = :dashdot)
plot!(pF_unstable_xvalues1, pF_unstable_yvalues1, label = "", linewidth = linewidth, color = :grey65)
plot!(pF_unstable_xvalues2, pF_unstable_yvalues2, label = "", linewidth = linewidth, color = :grey65)
P.plot_special_points(dF.γ, :μ4, :M; plot_legend = false)

xmin, xmax = 0.1, 6 
ymin, ymax = 0, BM.dimensional_M(1.6)

xlims!(xmin, xmax)
ylims!(ymin, ymax)

title!("\n b) control parameter F\$_\vphantom\$")
xlabel!("F [m yr\$^{-1}\$]")
ylabel!("M [Sv]")

savefig("Figures/bifdiagram-F.pdf")



ERROR: syntax error
ERROR: syntax error
ERROR: syntax error
ERROR: syntax error


"c:\\Users\\kvdhe\\Documents\\a. Studie\\6 - MSc 2\\Thesis\\Paper\\Code\\Figures\\bifdiagram-F.pdf"

In [9]:
# xmin, xmax = 0.1, 6 
# ymin, ymax = 0, BM.dimensional_M(1.6)

# pF = P.plot_branch(dF.γ , :μ4, :M, plot_legend = false,
#     xlabel = "F [m yr\$^{-1}\$]", ylabel = "M [Sv]", title = "b) Control parameter S\$_2\$", 
#     xlimits = (xmin, xmax), ylimits = (ymin, ymax))


# savefig("Figures/bifdiagram-F")

## Combined plot

In [63]:
# layout = @layout [a{0.4w} b{0.6w}]

plot(pS2, pF,  
    dpi = 300,
    ylimits = (14.5, 30),
    layout = (1, 2), 
    subtitlefontsize = 1,
    size = (width = 1000, height = 500), #(width = 800, height = 400),
    bottom_margin = 25px, left_margin = 20px, top_margin = 10px, right_margin = 20px)

savefig("Figures/bifdiagram.pdf")

ERROR: syntax error
ERROR: syntax error
ERROR: syntax error
ERROR: syntax error


"c:\\Users\\kvdhe\\Documents\\a. Studie\\6 - MSc 2\\Thesis\\Paper\\Code\\Figures\\bifdiagram.pdf"